In [1]:
from arg_utils import is_notebook, get_cfg
cfg = get_cfg()
# override variables to experiment in notebook
if is_notebook():
    cfg["gpu"] = 2
    cfg["debug"] = False
# overrides and calculated default vals
if cfg["batch_size"] is None:
    if cfg["model_name"] == "ViT-L/14":
        batch_size = 4 # max 32 for single GPU CL on VitB16, 4 for ViT-L/14 (9.2GB)
    elif cfg["model_name"] == "ViT-B/16":
        batch_size = 32
    elif cfg["model_name"] == "ViT-B/32":
        batch_size = 64
    else:
        batch_size = 32
    cfg["batch_size"] = batch_size
cfg["val_check_interval"] = int(cfg["val_check_interval"] * (32 / cfg["batch_size"]))
    
locals().update(cfg)

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu)

import pandas as pd
from PIL import Image
import numpy as np

import torch

root_folder = "/raid/8wiehe/"

In [3]:
from clip_utils import load_clip, FinetuneDataModule
from contrastive_learning_utils import LitCLCLIP

clip_base_model, transform, clip_name = load_clip(model_name, device="cpu")

data_module = FinetuneDataModule(clip_base_model, transform, dataset_name=dataset_name, mode=mode, 
                                 use_augs=use_augs, use_cl=True, sent_frac=sent_frac, batch_size=batch_size,
                                root_folder=root_folder, use_ffcv=use_ffcv)

lit_model = LitCLCLIP(clip_base_model, mode, max_epochs, lr, data_module.steps_per_epoch, 
                 weight_decay=weight_decay, gen_freq=gen_freq, use_ffcv=use_ffcv)
lit_model.label_names = data_module.label_names

In [4]:
for batch in data_module.train_dataloader():
    print(len(batch))
    break

3


In [5]:
len(data_module.val_dataloader())

93

In [6]:

import pytorch_lightning
from pytorch_lightning.loggers import WandbLogger

# for ffcv
if use_ffcv:
    from types import MethodType
    import ffcv_custom_PTL_methods 


if debug:
    wandb_logger = None
else:
    wandb_logger = pytorch_lightning.loggers.WandbLogger(name=None, 
                                                         save_dir=root_folder + "pytorch_lightning/", 
                                                         offline=False, id=None, 
                                          anonymous=None, version=None, project="cl_early_tests", 
                                          log_model=False, experiment=None, prefix='')
    wandb_logger.log_hyperparams(cfg)
    """
    wandb_logger.log_hyperparams({"mode": mode,
                                  "dataset_name": dataset_name,
                                  "sent_frac": sent_frac,
                                  "use_augs": use_augs,
                                  "batch_size": batch_size,
                                  "model_name": model_name,
                                  "use_ffcv": use_ffcv,
                                 })
    """
    # log gradients and model topology
    wandb_logger.watch(lit_model)


trainer = pytorch_lightning.Trainer(val_check_interval=val_check_interval,
                                    precision=precision,
                                    logger=wandb_logger,
                                    max_epochs=max_epochs,
                                    gpus=int(torch.cuda.is_available()),
                                    #overfit_batches=1, 
                                    benchmark=True,
                                    )

if use_ffcv:
    # for ffcv
    trainer.fit_loop.epoch_loop.on_run_start = MethodType(ffcv_custom_PTL_methods.on_run_start, trainer.fit_loop.epoch_loop)
    trainer.fit_loop.epoch_loop.advance = MethodType(ffcv_custom_PTL_methods.advance, trainer.fit_loop.epoch_loop)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: antonius (use `wandb login --relogin` to force relogin)


Unable to join threads to shut down before fork(). This can break multithreading in child processes.

Unable to join threads to shut down before fork(). This can break multithreading in child processes.

Unable to join threads to shut down before fork(). This can break multithreading in child processes.



wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
trainer.fit(lit_model, data_module)
# remove wandb hooks
#wandb_logger.unwatch(model)

/srv/home/8wiehe/miniconda3/envs/ffcv/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:104: UserWarning: attribute 'model' removed from hparams because it cannot be pickled
  rank_zero_warn(f"attribute '{k}' removed from hparams because it cannot be pickled")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [2]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | CLIP             | 149 M 
1 | loss_func | CrossEntropyLoss | 0     
-----------------------------------------------
81.9 M    Trainable params
67.7 M    Non-trainable params
149 M     Total params
299.241   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Unable to join threads to shut down before fork(). This can break multithreading in child processes.



Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
trainer.val_check_batch